In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

In [ ]:
# 1. Download Pakistan’s drivable road network
G = ox.graph_from_place("Pakistan", network_type="drive")

# 2. Load your district centroids
gdf = gpd.read_file("Districts_Pakistan.geojson") # Replace w Lahore
gdf['centroid'] = gdf.geometry.centroid

# 3. For each district, compute avg distance to nearest primary road
#    (as a simple proxy for accessibility)
def get_nearest_dist(pt):
    # find nearest node in graph
    u = ox.nearest_nodes(G, pt.x, pt.y)
    # get all edge lengths, take minimum
    lengths = [data['length'] for _, _, data in G.edges(u, data=True)]
    return min(lengths) if lengths else None

gdf['road_dist_m'] = gdf['centroid'].apply(get_nearest_dist)

# 4. Save results
gdf[['DISTRICT','road_dist_m']].to_csv("district_road_access.csv", index=False)


C:\Users\Ahmad Hassan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 562 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001C33661B310>>
Traceback (most recent call last):
  File "C:\Users\Ahmad Hassan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [8]:
from tqdm import tqdm
import osmnx as ox
import geopandas as gpd

# Load your GeoJSON districts
gdf = gpd.read_file("../Districts_Pakistan.geojson")

# Store result
all_graphs = {}

for i, row in tqdm(gdf.iterrows(), total=len(gdf)):
    district_name = row["DistrictCo"]
    geom = row["geometry"]

    try:
        G = ox.graph_from_polygon(geom, network_type="drive")
        all_graphs[district_name] = G
    except Exception as e:
        print(f"Failed for {district_name}: {e}")




  0%|          | 0/161 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [7]:
import os
import pandas as pd

# Create directories for exports
os.makedirs("graphs", exist_ok=True)
os.makedirs("shapefiles", exist_ok=True)
os.makedirs("stats", exist_ok=True)

# Export all graphs and create shapefiles with stats
district_stats = []

for district_name, G in tqdm(all_graphs.items(), desc="Exporting graphs and shapefiles"):
    try:
        # 1. Save GraphML file
        ox.save_graphml(G, filepath=f"graphs/{district_name}.graphml")
        
        # 2. Convert to GeoDataFrames and save as shapefiles
        # Nodes (intersections)
        nodes_gdf = ox.graph_to_gdfs(G, edges=False)
        nodes_gdf.to_file(f"shapefiles/{district_name}_nodes.shp")
        
        # Edges (roads)
        edges_gdf = ox.graph_to_gdfs(G, nodes=False)
        edges_gdf.to_file(f"shapefiles/{district_name}_edges.shp")
        
        # 3. Calculate network statistics
        stats = {
            'district': district_name,
            'num_nodes': len(G.nodes),
            'num_edges': len(G.edges),
            'total_length_km': edges_gdf['length'].sum() / 1000,
            'avg_edge_length_m': edges_gdf['length'].mean(),
            'max_edge_length_m': edges_gdf['length'].max(),
            'min_edge_length_m': edges_gdf['length'].min(),
            'network_density': len(G.edges) / len(G.nodes) if len(G.nodes) > 0 else 0
        }
        
        # Add connectivity metrics if graph has edges
        if len(G.edges) > 0:
            stats['avg_degree'] = sum(dict(G.degree()).values()) / len(G.nodes)
            stats['is_connected'] = ox.is_strongly_connected(G)
        else:
            stats['avg_degree'] = 0
            stats['is_connected'] = False
            
        district_stats.append(stats)
        
        print(f"✓ Exported {district_name}: {stats['num_nodes']} nodes, {stats['num_edges']} edges")
        
    except Exception as e:
        print(f"✗ Failed to export {district_name}: {e}")
        # Add error entry to stats
        district_stats.append({
            'district': district_name,
            'num_nodes': 0,
            'num_edges': 0,
            'total_length_km': 0,
            'avg_edge_length_m': 0,
            'max_edge_length_m': 0,
            'min_edge_length_m': 0,
            'network_density': 0,
            'avg_degree': 0,
            'is_connected': False,
            'error': str(e)
        })

NameError: name 'tqdm' is not defined

In [5]:
# Save comprehensive statistics
stats_df = pd.DataFrame(district_stats)
stats_df.to_csv("stats/district_road_network_stats.csv", index=False)

# Create summary report
print("\n" + "="*60)
print("EXPORT SUMMARY")
print("="*60)
print(f"Total districts processed: {len(all_graphs)}")
print(f"Successfully exported: {len([s for s in district_stats if s.get('num_edges', 0) > 0])}")
print(f"Failed exports: {len([s for s in district_stats if 'error' in s])}")

print(f"\nFiles created:")
print(f"- GraphML files: graphs/")
print(f"- Node shapefiles: shapefiles/*_nodes.shp")
print(f"- Edge shapefiles: shapefiles/*_edges.shp") 
print(f"- Statistics CSV: stats/district_road_network_stats.csv")

# Display top 10 districts by road network size
top_districts = stats_df.nlargest(10, 'total_length_km')[['district', 'total_length_km', 'num_nodes', 'num_edges']]
print(f"\nTop 10 districts by total road length:")
print(top_districts.to_string(index=False))

# Show basic statistics
print(f"\nNetwork Statistics Summary:")
print(f"Total road length across all districts: {stats_df['total_length_km'].sum():.2f} km")
print(f"Average road length per district: {stats_df['total_length_km'].mean():.2f} km")
print(f"Total road intersections: {stats_df['num_nodes'].sum():,}")
print(f"Total road segments: {stats_df['num_edges'].sum():,}")

print("\n" + "="*60)

NameError: name 'district_stats' is not defined

In [ ]:
import os
import osmnx as ox
import pandas as pd

# Folder where graphs are saved
graph_folder = "./graphs"

# List to store stats
stats_list = []

for fname in os.listdir(graph_folder):
    if fname.endswith(".graphml"):
        district = fname.replace(".graphml", "")
        try:
            G = ox.load_graphml(os.path.join(graph_folder, fname))
            stats = ox.basic_stats(G)
            stats["district"] = district
            stats_list.append(stats)
        except Exception as e:
            print(f"Error loading {district}: {e}")

df_stats = pd.DataFrame(stats_list)
df_stats.to_csv("district_road_network_stats.csv", index=False)
